### This is a python function code for Cluster-Wise Regression and Prediction
### Author: Duo Zhou
#### Reference: https://arxiv.org/pdf/1607.01417.pdf

GermanCredit dataset were used to test this function


In [436]:
import pandas as pd  
import numpy as np  
import statsmodels.formula.api as smf
import statsmodels.api as sm
import random
import scipy.stats
import warnings
import math
warnings.filterwarnings('ignore')

In [134]:
GC=pd.read_csv('GermanCredit.csv')
GC=GC.iloc[:,1:]
GC=GC.iloc[:,[1,0,2,3,4,5,6]]
GC.head()

,Amount,Duration,InstallmentRatePercentage,ResidenceDuration,Age,NumberExistingCredits,NumberPeopleMaintenance
0,1169,6,4,4,67,2,1
1,5951,48,2,2,22,1,1
2,2096,12,2,3,49,1,2
3,7882,42,2,4,45,1,2
4,4870,24,3,4,53,2,2


In [441]:
GC.drop([0,3,2]).head()

,Amount,Duration,InstallmentRatePercentage,ResidenceDuration,Age,NumberExistingCredits,NumberPeopleMaintenance
1,5951,48,2,2,22,1,1
4,4870,24,3,4,53,2,2
5,9055,36,2,4,35,1,2
6,2835,24,3,4,53,1,1
7,6948,36,2,2,35,1,1


In [465]:
nr=GC.shape[0]
train_ind = random.sample(range(nr),k=math.floor(0.7*nr))
train=GC.iloc[train_ind,:].reset_index()
test= GC.drop(train_ind).reset_index()
train=train.drop('index',axis=1)
test=test.drop('index',axis=1)

In [466]:
train.head()

,Amount,Duration,InstallmentRatePercentage,ResidenceDuration,Age,NumberExistingCredits,NumberPeopleMaintenance
0,8072,30,2,3,25,3,1
1,8358,48,1,1,30,2,1
2,1048,10,4,4,23,1,1
3,1386,12,2,2,26,1,1
4,14782,60,3,4,60,2,1


In [467]:
test.head()

,Amount,Duration,InstallmentRatePercentage,ResidenceDuration,Age,NumberExistingCredits,NumberPeopleMaintenance
0,1169,6,4,4,67,2,1
1,2096,12,2,3,49,1,2
2,7882,42,2,4,45,1,2
3,5234,30,4,2,28,2,1
4,1295,12,3,1,25,1,1


In [468]:
print(train.shape)
print(test.shape)

(700, 7)
(300, 7)


In [477]:
def clustreg(dat,clust,tries,sed,niter):
# make sure response variable is the first column of dat 
    random.seed(sed)
    dat=pd.DataFrame(dat)
    rsq=[None] * niter
    res=[]
    rsq_best=0
    nrows=dat.shape[0]
    formula = dat.columns[0] +'~'+'+'.join(dat.columns[1:]) 
    for l in range(tries):
        c = np.array(random.choices(range(clust),k=nrows))
        yhat = [None]*nrows
        for i in range(niter):
            pred=np.zeros([nrows,clust])
            resid=pd.DataFrame(0, index=np.arange(nrows), columns=np.arange(clust))
            for j in range(clust):
                model1 = smf.glm(formula = formula, data=dat.iloc[c==j,:], family=sm.families.Gaussian())
                result1 = model1.fit()
                pred[:,j]=result1.predict(dat.iloc[:,1:])
                resid.iloc[:,j] =(pred[:,j]-dat.iloc[:,0])**2
            c=np.array(resid.apply(np.argmin, axis=1))
            for m in range(nrows):
                yhat[m]=pred[m,c[m]]
            rsq[i]= (scipy.stats.pearsonr(yhat,dat.iloc[:,0])[0])**2
        if rsq[niter-1] > rsq_best:
            rsq_best=rsq[niter-1]
            l_best=l
            c_best=c
            yhat_best=yhat
    return  {'Formula':formula,'Best R^2':rsq_best,'Number of Clusters':clust,'Best Clustering':c_best}
                

In [478]:
resl = clustreg(dat=train,clust=3,tries=2,sed=1254,niter=10)
resl

{'Formula': 'Amount~Duration+InstallmentRatePercentage+ResidenceDuration+Age+NumberExistingCredits+NumberPeopleMaintenance',
 'Best R^2': 0.8858476985073005,
 'Number of Clusters': 3,
 'Best Clustering': array([0, 1, 2, 2, 1, 2, 0, 2, 1, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 1, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2, 0, 2,
        2, 2, 0, 0, 2, 2, 2, 0, 1, 2, 2, 2, 1, 0, 2, 2, 0, 2, 2, 0, 2, 2,
        0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 0, 1, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2,
        2, 0, 1, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 0, 1, 1, 2, 2, 2, 0, 0, 2,
        2, 0, 2, 2, 1, 0, 2, 0, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 0, 2, 2,
        2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 1,
        2, 2, 2, 1, 2, 0, 2, 2, 1, 0, 2, 0, 2, 0, 2, 1, 2, 0, 2, 0, 1, 2,
        2, 0, 2, 2, 2, 2, 0, 2, 1, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1,
        2, 2, 0, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 2, 0,
        0, 2, 0, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 0, 2

In [479]:
def clustreg_predict(results,newdat,olddat):
# make sure response variable is the first column of newdat and olddat 
    nrows=newdat.shape[0]
    nclust=results['Number of Clusters']
    yhat=[None]*nrows
    c=results['Best Clustering']
    formula = olddat.columns[0] +'~'+'+'.join(olddat.columns[1:]) 
    pred=np.zeros([nrows,nclust])  
    resid=pd.DataFrame(0, index=np.arange(nrows), columns=np.arange(nclust))
    for j in range(nclust):
        model1 = smf.glm(formula = formula, data=olddat.iloc[c==j,:], family=sm.families.Gaussian())
        result1 = model1.fit()
        pred[:,j]=result1.predict(newdat.iloc[:,1:])        
        resid.iloc[:,j] =(pred[:,j]-newdat.iloc[:,0])**2
    c=np.array(resid.apply(np.argmin, axis=1))
    for m in range(nrows):
        yhat[m]=pred[m,c[m]]
    rsq = (scipy.stats.pearsonr(yhat,newdat.iloc[:,0])[0])**2 
    
    return {'Formula':formula,'Best R^2':rsq,'yhat':yhat,'Best Clustering':c}

In [480]:
pred = clustreg_predict(results=resl,newdat=test, olddat=train)
pred

{'Formula': 'Amount~Duration+InstallmentRatePercentage+ResidenceDuration+Age+NumberExistingCredits+NumberPeopleMaintenance',
 'Best R^2': 0.8716012406948798,
 'yhat': [461.76473202527404,
  1781.846863183681,
  10463.467835253894,
  2733.9725594391984,
  1249.7156434067463,
  2164.2326784420657,
  1959.170144857934,
  984.2752600891122,
  1476.1790316345332,
  3118.611325568054,
  2830.0172037101765,
  1189.4294630792065,
  5532.327180022587,
  595.0934976376996,
  2526.0109495834204,
  1247.770111166414,
  10342.67695238185,
  1822.1957058694734,
  1965.0067415789306,
  5857.9896695829475,
  4246.106625669784,
  1394.9094076289612,
  1728.604585273566,
  12858.574745729278,
  540.9789748414003,
  3124.9722583632415,
  1228.770510673845,
  3671.8617465808957,
  3374.9611596874984,
  1964.0531975476165,
  1092.7366959103167,
  845.9358460193021,
  13369.470799302226,
  1447.7962310707699,
  716.4448808714178,
  6960.615068089154,
  7869.794593030587,
  2306.7630775381062,
  1860.8135019

In [482]:
resl1 = clustreg(dat=train,clust=1,tries=1,sed=1254,niter=1)
resl2 = clustreg(dat=train,clust=2,tries=2,sed=1254,niter=10)
resl3 = clustreg(dat=train,clust=3,tries=2,sed=1254,niter=10)
pred1 = clustreg_predict(results=resl1,newdat=test, olddat=train)
pred2 = clustreg_predict(results=resl2,newdat=test, olddat=train)
pred3 = clustreg_predict(results=resl3,newdat=test, olddat=train)

rsq_b_train1=resl1['Best R^2']
rsq_b_train2=resl2['Best R^2']
rsq_b_train3=resl3['Best R^2']
rsq_b_test1=pred1['Best R^2']
rsq_b_test2=pred2['Best R^2']
rsq_b_test3=pred3['Best R^2']

table=pd.DataFrame(index=['Model 1', 'Model 2','Model 3'], columns=['R^2 Train', 'R^2 Test'])

table.iloc[0,0]=rsq_b_train1
table.iloc[0,1]=rsq_b_test1
table.iloc[1,0]=rsq_b_train2
table.iloc[1,1]=rsq_b_test2
table.iloc[2,0]=rsq_b_train3
table.iloc[2,1]=rsq_b_test3

table

,R^2 Train,R^2 Test
Model 1,0.477095,0.548354
Model 2,0.805673,0.813802
Model 3,0.885848,0.871601
